In [ ]:
import signal
import threading
from typing import Dict

from ipyleaflet import Circle, LayerGroup, Map
from visionapi.messages_pb2 import SaeMessage
from visionlib.pipeline.consumer import RedisConsumer


# Map setup
CENTER = (39.95871094145749, -86.12691605973491)

m = Map(center=CENTER, zoom=18)
marker_layer = LayerGroup()
m.add(marker_layer)
display(m)

displayed_objects: Dict[str, Circle] = dict()


# Setup graceful exit on SIGINT
stop_event = threading.Event()

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)


# Start listening to SAE messages
consume = RedisConsumer('localhost', 6379, ['geomapper:stream1'], block=200)

with consume:
    for stream_key, proto_data in consume():

        if stop_event.is_set():
            break

        if stream_key is None:
            continue

        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)

        # Remove objects from cache and map that were rendered before but do not exist anymore
        removed_objects = list(filter(lambda id: id not in map(lambda d: d.object_id, sae_msg.detections), displayed_objects.keys()))
        for id in removed_objects:
            removed = displayed_objects.pop(id, None)
            if removed is not None:
                marker_layer.remove(removed)

        # Add new objects / Update existing objects locations
        for detection in sae_msg.detections:
            lat, lon = detection.geo_coordinate.latitude, detection.geo_coordinate.longitude
            if detection.object_id in displayed_objects:
                displayed_objects[detection.object_id].location = (lat, lon)
            else:
                marker = Circle(location=[lat, lon], radius=2)
                displayed_objects[detection.object_id] = marker
                marker_layer.add(marker)